# Data Prep
In order to get start the EDA process, our data sources must be merged and cleaned. This file is for that purpose. Multiple data sources will be used to merge the data properly.
- Pricing Data from the VA. (https://www.va.gov/oalc/foia/library.asp#two)
- Historical Consumer Price index. (https://www.usinflationcalculator.com/inflation/consumer-price-index-and-annual-percent-changes-from-1913-to-2008/)
- Evergreen Patent Database from UC Hastings. (https://sites.uchastings.edu/evergreensearch/archive/)
- FDA approved drugs list.(https://www.fda.gov/drugs/development-approval-process-drugs/national-drug-code-database-background-information)

**Note** VA pricing data has already been pre-cleaned for use by a different file.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Importing Data

In [2]:
#Dataframe for Evergreen Database
Evgn = pd.read_csv("EvergreenDatasetRaw_Dataset_2005-2018_v02.csv")
#Dataframe for individual drugs from Orange Book
#Product = pd.read_csv('products.txt',delimiter="~")
#Dataframe for individual patents from Orange Book
#Patent = pd.read_csv("patent.txt",delimiter="~")
#Dataframe for linking NDA (New Drug Application Number) to NDC (National Drug Code)
Link = pd.read_csv("20220906_product.csv")
#Dataframe for Historical VA Pricing. Please download the csv from the following link and change the dir you need.
#https://drive.google.com/uc?id=1ZWlyvtMNjOrOtFlFotvnPDoKeaBD0voe
Pricing = pd.read_csv("C:\\Users\\sulli\\Documents\\Code\\Studio 4\\Drug Patents\\historicvapricing.csv")
#Dataframe for CPI. Needed to adjust historical pricing for inflation.
CPI = pd.read_csv("CPI.csv")

C:\Users\sulli\AppData\Local\Temp\ipykernel_22832\2086762004.py:11: DtypeWarning: Columns (6,13) have mixed types. Specify dtype option on import or set low_memory=False.
  Pricing = pd.read_csv("C:\\Users\\sulli\\Documents\\Code\\Studio 4\\Drug Patents\\historicvapricing.csv")


## Subsetting Historical Pricing
In order to properly link the VA pricing to the Evergreen patent database, the FDA approved drugs list will be needed to link the NDA to the NDC. The following code block subsets the "Link" dataframe such that only the approved drugs found in the Evergreen database exist.

In [3]:
Link = Link[['applicationnumber','productndc']]
Link = Link[~(Link['applicationnumber'].isna())]
Link = Link[Link['applicationnumber'].str.contains('NDA')]
Link['applicationnumber'] = Link['applicationnumber'].apply(lambda x: (x.replace('ANDA','')))
Link['applicationnumber'] = Link['applicationnumber'].apply(lambda x: int(x.replace('NDA','')))
Link = Link[Link['applicationnumber'].isin(Evgn['NDA #'])]

In order to properly link the NDC code in the the "Link" dataframe to that of the historical pricing, the NDC code needs to be clean. The following code block does this.

In [4]:
Pricing[['NDC1','NDC2','NDC3']] =  Pricing.NDCWithDashes.str.split('-',expand=True)
Pricing=Pricing.drop(columns=['NDC3'])
Pricing['CleanNDC'] = Pricing['NDC1'] + '-' + Pricing['NDC2']

Finally the "Pricing" dataframe is subset using the NDC and merged with the "Link" dataframe using an inner join.

In [5]:
Pricing = Pricing[Pricing['CleanNDC'].isin(Link['productndc'])]
Pricing = Pricing.merge(Link, how='inner',left_on='CleanNDC',right_on='productndc')

The "Link" dataframe will be deleted to save memory.

In [6]:
del(Link)

## Adjusting Historical Pricing for Inflation
Because we are working with historical prices over time, the data will neeed to be adjusted for inflation. The CPI will be used to account for this. First the month and year of the price start date will need to be retrieved.

In [7]:
Pricing['PriceStartDate'] = pd.to_datetime(Pricing['PriceStartDate'])
Pricing['StartMonth'] = Pricing['PriceStartDate'].dt.month
Pricing['StartYear'] = Pricing['PriceStartDate'].dt.year 

Next the CPI dataframe will need to be change into the long form in order to create a proper merge.

In [8]:
CPI_numbers = []
CPI_month = []
CPI_year = []
month_year = {1:'Jan',2:'Feb',3:'Mar',4:'Apr',
                 5:'May',6:'June',7:'July',8:'Aug',
                 9:'Sep',10:'Oct',11:'Nov',12:'Dec'}
for year in [2005,2006,2007,2008,2009,2010,2011,
            2012,2013,2014,2015,2016,2017,2018]:
    for month in [1,2,3,4,5,6,7,8,9,10,11,12]:
        column = month_year[month]
        CPI_year.append(year)
        CPI_month.append(month)
        CPI_numbers.append(float(CPI[CPI['Year']==year][column]))

CPI = pd.DataFrame(data=zip(CPI_numbers,CPI_month,CPI_year),
                  columns=['CPI','Month','Year'])

Pricing will then be merged with the CPI. 

In [9]:
Pricing = Pricing.merge(CPI, how='inner',left_on=['StartMonth','StartYear'],
                       right_on=['Month','Year'])

Finally, a new column called "InflationAdjustedPrice" will be created.

In [10]:
Pricing['Price'] = pd.to_numeric(Pricing['Price'])
Pricing['InflationAdjustedPrice'] = (Pricing['Price'])/(Pricing['CPI']) * 100

## Reducing Number of Columns
Many columns that have been engineered are now useless and serve as fluff. They will be dropped.

In [11]:
Pricing = Pricing.drop(columns=['Unnamed: 0','NDC1','NDC2','NDCWithDashes',
                               'Month','Year','DateKey'])

In [12]:
Pricing.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65930 entries, 0 to 65929
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   ContractNumber          65930 non-null  object        
 1   VendorName              65930 non-null  object        
 2   ContractStartDate       65930 non-null  object        
 3   ContractStopDate        65930 non-null  object        
 4   SubItemIdentifier       59196 non-null  object        
 5   PackageDescription      65930 non-null  object        
 6   Generic                 65930 non-null  object        
 7   TradeName               65930 non-null  object        
 8   VAClass                 65930 non-null  object        
 9   Covered                 65930 non-null  object        
 10  PrimeVendor             65930 non-null  object        
 11  Price                   65930 non-null  float64       
 12  PriceStartDate          65930 non-null  dateti

## Adding Features to Evergreen Dataframe
There are some features that would be beneficial inclusions in the Evergreen Dataframe. Particularly, including the previous number of patents for a given drug would be a good start. First, the "Date Added" column will have to be change for any drug listed as "pre-2005". For ease of use, they will be entered as "1/1/05".

In [13]:
Evgn

,Active Ingredient,NDA #,Product Name,Company,Approval Date,P or E,Date Added,Patent Number,Expiration Date,Codes,Strengths,Delist Request,Orig,Analysis,Added strength,# added strengths,Applied to UC,2nd add,Comments
0,Abacavir Sulfate,20977,Ziagen*,VIIV HLTHCARE,12/17/98,P,pre-2005,5034394,12/18/11,NaN,1,NaN,NaN,Pre-2005,NaN,NaN,NaN,NaN,10/26/72
1,Abacavir Sulfate,20977,Ziagen*,VIIV HLTHCARE,NaN,P,pre-2005,5089500,6/26/09,U-248,1,NaN,NaN,Pre-2005,NaN,NaN,NaN,NaN,NaN
2,Abacavir Sulfate,20977,Ziagen*,VIIV HLTHCARE,NaN,P,pre-2005,6294540,5/14/18,U-65,1,NaN,NaN,Pre-2005,NaN,NaN,NaN,NaN,NaN
3,Abacavir Sulfate,20977,Ziagen*,VIIV HLTHCARE,NaN,P,pre-2005,5034394*PED,6/18/12,NaN,1,NaN,NaN,Pre-2005,NaN,NaN,NaN,NaN,12/22/15
4,Abacavir Sulfate,20977,Ziagen*,VIIV HLTHCARE,NaN,P,pre-2005,5089500*PED,12/26/09,NaN,1,NaN,NaN,Pre-2005,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21287,Zolpidem Tartrate,22328,Intermezzo,PURDUE PHARMA,NaN,P,11/28/11,7682628,2/16/25,U-1194,"001, 002",NaN,Yes,P:UC,NaN,NaN,NaN,NaN,NaN
21288,Zolpidem Tartrate,22328,Intermezzo,PURDUE PHARMA,NaN,E,11/28/11,NaN,11/23/14,NP,"001, 002",NaN,Yes,NP,NaN,NaN,NaN,NaN,NaN
21289,Zolpidem Tartrate,22328,Intermezzo,PURDUE PHARMA,NaN,P,8/28/12,8242131,8/20/29,U-1266,"001, 002",NaN,No,P:UCnew,NaN,NaN,NaN,NaN,NaN
21290,Zolpidem Tartrate,22328,Intermezzo,PURDUE PHARMA,NaN,P,8/28/12,8252809,2/16/25,DP,"001, 002",NaN,No,P:DP,NaN,NaN,NaN,NaN,NaN


In [14]:
Evgn['Pre2005Flag'] = [0] *len(Evgn)
Evgn.loc[Evgn['Date Added']=='pre-2005','Pre2005Flag']=1
Evgn.loc[Evgn['Date Added']=='pre-2005','Date Added']= '1/1/05' 

The dates for "Date Added" and "Expiration Date" need to be properly formatted. Currently the year is listed as the last two digits. To fix this we can simply add 2000 years to the date and cast it as date time.

In [15]:
Evgn['AddedMonth'] = Evgn['Date Added'].apply(lambda x: x.split('/')[0])
Evgn['AddedDay'] = Evgn['Date Added'].apply(lambda x: x.split('/')[1])
Evgn['AddedYear'] = Evgn['Date Added'].apply(lambda x: x.split('/')[2])
#This handles a specific outlier that messed things up
Evgn.loc[Evgn['AddedYear'].str.len() == 1,'AddedYear'] = '0' + Evgn['AddedYear']
Evgn['Date Added'] = pd.to_datetime(Evgn['AddedMonth'] + '/' + 
                                    Evgn['AddedDay'] + '/20' + Evgn['AddedYear'])
Evgn['ExpirationMonth'] = Evgn['Expiration Date'].apply(lambda x: x.split('/')[0])
Evgn['ExpirationDay'] = Evgn['Expiration Date'].apply(lambda x: x.split('/')[1])
Evgn['ExpirationYear'] = Evgn['Expiration Date'].apply(lambda x: x.split('/')[2])
Evgn['Expiration Date'] = pd.to_datetime(Evgn['ExpirationMonth'] + '/' + 
                                         Evgn['ExpirationDay'] + '/20' + Evgn['ExpirationYear'])
Evgn = Evgn.drop(columns=['AddedMonth','AddedDay','AddedYear','ExpirationMonth','ExpirationDay','ExpirationYear'])

Now the complicated part will be to do math based on previous information. This will be a slow for loop that will hopefully run not terribly slow.

In [16]:
#Used for index in for loop
Evgn['Indexer'] = Evgn.index
PreviousPatents = []
nextExpiration = []
for index in Evgn['Indexer']:
    dateAdded = Evgn[Evgn['Indexer']==index]['Date Added']
    drugNDA = Evgn[Evgn['Indexer']==index]['NDA #']
    subset2 = Evgn[(Evgn['NDA #']==drugNDA.values[0])&
                             (Evgn['Date Added']<dateAdded.values[0])]
    count_previous = len(subset2)
    PreviousPatents.append(count_previous)
    
    nextExpiration.append(Evgn[(Evgn['NDA #']==drugNDA.values[0])&
                             (Evgn['Date Added']<=dateAdded.values[0])]["Expiration Date"].max())
    

In [17]:
Evgn['PreviousPatents'] = PreviousPatents
Evgn['LatestExpiration'] = nextExpiration
Evgn=Evgn.drop(columns='Indexer')

## Merging Evergreen Database with Pricing
The final step in our dataprep phase is to get one concrete dataset. To accomplish this, the evergreen dataset will be merged with the pricing dataset using an inner join on the "NDA #". 

In [18]:
Pricing = Pricing.drop_duplicates()
Pricing['Indexer'] = Pricing.index
Combined = Pricing.merge(Evgn,how='inner',left_on="applicationnumber",
                        right_on="NDA #")

Because an NDA can appear multiple times, logic needs to be included such that only the most recent patent is included. This means two things, patents with an addition date later than the pricing should be excluded and only the most recent patent should be included.

In [19]:
Combined = Combined[Combined['Date Added'] <= Combined['PriceStartDate']]

The following codeblock retrieves the most recent patent information and subset's to the correct records.

In [20]:
Combined['Indexer2'] = Combined.index
mostRecentAddition = []
for index in Combined['Indexer'].unique():
    subset = Combined[Combined['Indexer']==index].sort_values("Date Added",ascending=False)
    mostRecentAddition.append(list(subset['Indexer2'])[0])
Combined=Combined[Combined['Indexer2'].isin(mostRecentAddition)]

In [21]:
Combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6161 entries, 12 to 112616
Data columns (total 46 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   ContractNumber          6161 non-null   object        
 1   VendorName              6161 non-null   object        
 2   ContractStartDate       6161 non-null   object        
 3   ContractStopDate        6161 non-null   object        
 4   SubItemIdentifier       4693 non-null   object        
 5   PackageDescription      6161 non-null   object        
 6   Generic                 6161 non-null   object        
 7   TradeName               6161 non-null   object        
 8   VAClass                 6161 non-null   object        
 9   Covered                 6161 non-null   object        
 10  PrimeVendor             6161 non-null   object        
 11  Price                   6161 non-null   float64       
 12  PriceStartDate          6161 non-null   datet

Finally, we'll grab all the columns that will be used for the analysis.

In [22]:
Combined = Combined[['Price','PriceStartDate','Date Added',
                     'InflationAdjustedPrice','Analysis',
                     'P or E','Pre2005Flag','PreviousPatents','LatestExpiration']]
Combined = Combined.reset_index().drop(columns='index')

Now the dataframe will be exported to a csv file for later use.

In [23]:
Combined.to_csv("CleanedData.csv",index=False)